In [ ]:
!pip install pandas

Defaulting to user installation because normal site-packages is not writeable


In [12]:
import re
from pyspark.sql import SparkSession
import subprocess
import pandas as pd
from datetime import date

In [13]:
spark = SparkSession \
    .builder \
    .master("local[4]") \
    .appName("Caching") \
    .config("spark.sql.warehouse.dir", "file:///data") \
    .enableHiveSupport() \
    .getOrCreate()

sc = spark.sparkContext

In [14]:
today = date.today().strftime("%Y-%m-%d")

In [15]:
filename = f"b2b_{today}.csv"

In [ ]:
daily=spark.sql("""
Select s.name, p.product_name, sum(f.units) as count_of_units
From sales_schema.dim_product as p
Join sales_schema.super_fact as f On p.product_id = f.product_id
Join sales_schema.dim_sales_agent as s On s.sales_person_id = f.sales_agent_id
Group by s.name, p.product_name
Order by count_of_units desc
""")


In [17]:
daily.show()

+------------------+---------------+--------------+
|              name|   product_name|count_of_units|
+------------------+---------------+--------------+
|      Olivia Davis|       Sneakers|          4148|
|   Daniel Martinez|          Boots|          4134|
|       Emma Taylor|        Blender|          3172|
|      Olivia Davis|        Sandals|          3124|
|        Jane Smith|     Headphones|          3038|
|       Emma Taylor|          Boots|          2956|
|Christopher Miller|          Boots|          2888|
|      Sophia Moore|        Printer|          2754|
|Christopher Miller|        Toaster|          2552|
|      Sophia Moore|     Headphones|          2542|
|        Jane Smith|     Smartphone|          2350|
|        Jane Smith|          Boots|          2298|
|        Jane Smith|        Toaster|          2262|
|          John Doe|          Boots|          2201|
|      Sophia Moore|          Boots|          2155|
|       Emily Brown|        Blender|          2135|
|      David

In [ ]:
daily.coalesce(1) \
  .write.format('csv') \
  .option('header', True) \
  .mode('overwrite') \
  .option('sep', ',') \
  .save(f'file:///data/test/{filename}')

In [ ]:
# Define HDFS path and local destination path
#hdfs_output_path = 'hdfs:///data/test/output.csv'
#local_destination_path = '/Downloads/output.csv'


In [ ]:
# Use the Hadoop fs -get command to move the file to the local file system
#subprocess.run(['hadoop', 'fs', '-get', hdfs_output_path, local_destination_path])

CompletedProcess(args=['hadoop', 'fs', '-get', 'hdfs:///data/test/output.csv', '/Downloads/output.csv'], returncode=1)

In [32]:
#print(filename)

o_2024-07-11.csv


In [33]:
#print(today)

2024-07-11


In [ ]:
daily.coalesce(1) \
  .write.format('csv') \
  .option('header', True) \
  .mode('overwrite') \
  .option('sep', ',') \
  .save('/Downloads/filename')